In [ ]:
import os
import json

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [50]:
# obtain skeletons + label

SKELETON_PATH = r"C:\Users\Ryan\VSCode Projects\CV_gesture_detection\skeletons_tensor.pt" # get path
META_PATH = r"C:\Users\Ryan\VSCode Projects\CV_gesture_detection\skeleton_annots.json"# get path
skeletons = torch.load(SKELETON_PATH)  
print("Skeletons shape:", skeletons.shape)

with open(META_PATH, "r") as f:
    metadata = json.load(f)

labels = torch.tensor(
    [sample["label_id"] for sample in metadata["samples"]],
    dtype=torch.long,
)

# sanity check stuff
print("Labels shape:", labels.shape)
print("Num classes:", len(torch.unique(labels)))

Skeletons shape: torch.Size([3881, 210, 21, 3])
Labels shape: torch.Size([3881])
Num classes: 14


In [51]:
# training stuff 
num_samples = skeletons.shape[0]
indices = torch.randperm(num_samples)
split = int(0.8 * num_samples) #80-20 split
train_idx = indices[:split]
test_idx   = indices[split:]

train_skeletons = skeletons[train_idx]
train_labels    = labels[train_idx]

test_skeletons = skeletons[test_idx]
test_labels    = labels[test_idx]

train_ds = TensorDataset(train_skeletons, train_labels)
test_ds   = TensorDataset(test_skeletons, test_labels)

batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader   = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

print("Train samples:", len(train_ds))
print("Test samples:", len(test_ds))

Train samples: 3104
Test samples: 777


In [52]:
T, J, C = skeletons.shape[1:]  
input_dim = T * J * C
num_classes = len(torch.unique(labels)) + 1

print(num_classes)
class FlattenedCheeks(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 420),
            nn.ReLU(),
            nn.Linear(420, 67),
            nn.ReLU(),
            nn.Linear(67, num_classes),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1) 
        return self.net(x)

device = torch.device("cpu")
model = FlattenedCheeks(input_dim, num_classes).to(device)
print(model)

15
FlattenedCheeks(
  (net): Sequential(
    (0): Linear(in_features=13230, out_features=420, bias=True)
    (1): ReLU()
    (2): Linear(in_features=420, out_features=67, bias=True)
    (3): ReLU()
    (4): Linear(in_features=67, out_features=15, bias=True)
  )
)


In [53]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# training loop
def run_epoch(loader, model, criterion, optimizer=None):
    if optimizer is None:
        model.eval()
    else:
        model.train()

    total_loss = 0.0
    correct = 0
    total = 0

    for x_batch, y_batch in loader:
        x_batch = x_batch.to(device, dtype=torch.float32)
        y_batch = y_batch.to(device)

        if optimizer is not None:
            optimizer.zero_grad()

        logits = model(x_batch)           # forward pass
        loss = criterion(logits, y_batch) # compute loss

        if optimizer is not None:
            loss.backward()               # backprop
            optimizer.step()              # update weights

        total_loss += loss.item() * x_batch.size(0)

        preds = logits.argmax(dim=1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    avg_loss = total_loss / total
    acc = correct / total
    return avg_loss, acc

num_epochs = 10

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = run_epoch(train_loader, model, criterion, optimizer)
    test_loss, test_acc = run_epoch(test_loader, model, criterion, optimizer=None)

    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={train_loss:.4f}, train_acc={train_acc:.3f} | "
        f"test_loss={test_loss:.4f}, test_acc={test_acc:.3f}"
    )

Epoch 01 | train_loss=2.2684, train_acc=0.303 | test_loss=2.1089, test_acc=0.346
Epoch 02 | train_loss=2.0335, train_acc=0.362 | test_loss=1.9942, test_acc=0.369
Epoch 03 | train_loss=1.9380, train_acc=0.394 | test_loss=1.9033, test_acc=0.414
Epoch 04 | train_loss=1.7867, train_acc=0.454 | test_loss=1.7097, test_acc=0.450
Epoch 05 | train_loss=1.5999, train_acc=0.502 | test_loss=1.5562, test_acc=0.515
Epoch 06 | train_loss=1.4795, train_acc=0.528 | test_loss=1.4299, test_acc=0.524
Epoch 07 | train_loss=1.3298, train_acc=0.585 | test_loss=1.2638, test_acc=0.606
Epoch 08 | train_loss=1.2496, train_acc=0.603 | test_loss=1.1318, test_acc=0.624
Epoch 09 | train_loss=1.1351, train_acc=0.633 | test_loss=1.1917, test_acc=0.619
Epoch 10 | train_loss=1.0426, train_acc=0.659 | test_loss=1.1093, test_acc=0.611
